In [820]:
# importeer nodige libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

__INTERNE RED WINES DATASET__

In [821]:
# lees het bestand in
redwines = pd.read_csv("redwine.csv", delimiter=";", encoding='iso-8859-1')

# verwijder onnodige kolommen
redwines.drop(['country','variety','id'],axis=1, inplace=True)

# voeg jaartal toe als aparte kolom, en verwijder nan rijen voor jaren (2465 naar 2461)
redwines['year'] = redwines['title'].str.extract(r"([1][9][9]\d|[2][0][0,1,2]\d)")
redwines = redwines[redwines['year'].notnull()]
redwines['year'] = redwines['year'].astype(int)
redwines.set_index('year', inplace=True)

# drop rows with nan value in it
redwines.dropna(inplace=True)

__EXTERNE WEER DATASET__

In [822]:
# lees de externe dataset in
weather_conditions = pd.read_csv("weatherdata_lisbon.csv", delimiter=";", encoding='iso-8859-1')

# drop onbruikbare rijen en hernoem kolommen
weather_conditions.drop(['STN---','Unnamed: 5','Unnamed: 7','GUST','PRCP','SNDP','VISIB'], axis=1,inplace=True)
weather_conditions.dropna(inplace=True)
weather_conditions.rename(columns={'YEARMODA': 'Year','DEWP':'Dewpoint Temperature','TEMP': 'Temperature','MAX':'Max Temperature','MIN':'Min Temperature','WDSP':'Windspeed','MXSPD':'Max Windspeed'},inplace=True)

# cleanup and transform data types
# also convert fahrenheit to celsius
weather_conditions['Max Temperature'] = (weather_conditions['Max Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Min Temperature'] = (weather_conditions['Min Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Temperature'] = (weather_conditions['Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Dewpoint Temperature'] = (weather_conditions['Dewpoint Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Windspeed'] = weather_conditions['Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Max Windspeed'] = weather_conditions['Max Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Year']= pd.to_datetime(weather_conditions['Year'].astype(str), format='%Y-%m-%d')

# FRSHTT = Frost, Rain, Snow, Hail, Thunder, Thornado
weather_conditions['FRSHTT'] = weather_conditions['FRSHTT'].apply(lambda x: '{0:0>6}'.format(x))
weather_conditions['Frost'] = weather_conditions['FRSHTT'].str[0].astype(float)
weather_conditions['Rain'] = weather_conditions['FRSHTT'].str[1].astype(float)
weather_conditions['Snow'] = weather_conditions['FRSHTT'].str[2].astype(float)
weather_conditions['Hail'] = weather_conditions['FRSHTT'].str[3].astype(float)
weather_conditions['Thunder'] = weather_conditions['FRSHTT'].str[4].astype(float)
weather_conditions['Tornado'] = weather_conditions['FRSHTT'].str[5].astype(float)
weather_conditions.drop('FRSHTT',axis=1,inplace=True)

# verwijder uitschieters
weather_conditions = weather_conditions[weather_conditions['Min Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Dewpoint Temperature'] < 100]

# bereken gemiddelde eigenschappen per jaar
weather_conditions['Count'] = 1
average_weather_conditions_peryear = weather_conditions.groupby(temperatures['Year'].map(lambda x: x.year)).sum()
average_weather_conditions_peryear = average_weather_conditions_peryear[['Temperature','Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder','Tornado']].div(average_weather_conditions_peryear['Count'], axis=0)
average_weather_conditions_peryear.drop(pd.Int64Index([2018]), inplace=True) # 2018 heeft maar 1 meting

__MERGING EXTERNE EN INTERNE DATASETS__

In [823]:
redwines_year_comparison = redwines[['points']]
redwines_with_yeartemperatures = redwines_year_comparison.merge(average_weather_conditions_peryear, left_index=True, right_index=True)
redwines_with_yeartemperatures.head()

,points,Temperature,Dewpoint Temperature,Windspeed,Max Windspeed,Max Temperature,Min Temperature,Frost,Rain,Snow,Hail,Thunder,Tornado
1996,86,17.137675,12.306163,7.124863,13.982514,21.494536,13.598361,0.054645,0.385246,0.002732,0.000000,0.057377,0.0
2000,87,16.828931,11.736035,8.412568,14.268579,21.289617,13.341530,0.087432,0.338798,0.005464,0.002732,0.032787,0.0
2000,87,16.828931,11.736035,8.412568,14.268579,21.289617,13.341530,0.087432,0.338798,0.005464,0.002732,0.032787,0.0
2002,90,16.877473,12.720700,8.455616,14.497260,21.106849,13.508828,0.120548,0.364384,0.000000,0.000000,0.043836,0.0
2003,89,17.255403,12.032877,8.260548,14.665753,21.569863,13.835616,0.098630,0.326027,0.000000,0.000000,0.038356,0.0


__CREATING THE MODEL__

In [824]:
# ['Temperature', 'Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder','Tornado']
# dependent and independent columns
dependent = redwines_with_yeartemperatures[['points']]
independent = redwines_with_yeartemperatures.drop('points', axis=1)
#print(independent.head())

# test and train set
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.2)

# setting up and training model
model = LogisticRegression()
model.fit(X_train, y_train)

# testing and calculating accuracy
#print(model.predict(X_test))
print(model.score(X_test,y_test))

# LinearRegression 8%
# DecisionTreeClassifier 12%
# LogisticRegression 15%
# BayesianRidge 6%
# SVR 5%
# DecisionTreeRegressor 7%
# Ridge 5%

0.1553398058252427


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
